<H3>Importing Required Libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


<H3>Getting Spark Session

In [2]:
spark = SparkSession.builder.getOrCreate()

<H3>Reading CSV

In [3]:
df = spark.read.csv("Big_Cities_Health_Data_Inventory.csv", header=True)

In [4]:
df.show(10)

+------------------+--------------------+----+------+---------------+-----+--------------------+--------------------------+--------------------+-------+-----+
|Indicator Category|           Indicator|Year|Gender|Race/ Ethnicity|Value|               Place|BCHC Requested Methodology|              Source|Methods|Notes|
+------------------+--------------------+----+------+---------------+-----+--------------------+--------------------------+--------------------+-------+-----+
|          HIV/AIDS|AIDS Diagnoses Ra...|2013|  Both|            All| 30.4|Atlanta (Fulton C...|      AIDS cases diagno...|Diagnoses numbers...|   null| null|
|          HIV/AIDS|AIDS Diagnoses Ra...|2012|  Both|            All| 39.6|Atlanta (Fulton C...|      AIDS cases diagno...|Diagnoses numbers...|   null| null|
|          HIV/AIDS|AIDS Diagnoses Ra...|2011|  Both|            All| 41.7|Atlanta (Fulton C...|      AIDS cases diagno...|Diagnoses numbers...|   null| null|
|            Cancer|All Types of Canc...|2013|

<H3>Printing Schema

In [5]:
df.printSchema()

root
 |-- Indicator Category: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Race/ Ethnicity: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Place: string (nullable = true)
 |-- BCHC Requested Methodology: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Methods: string (nullable = true)
 |-- Notes: string (nullable = true)



<H3>Dropping Unwanted Columns

In [6]:
df = df.drop("Notes", "Methods", "Source", "BCHC Requested Methodology")

In [7]:
df.printSchema()

root
 |-- Indicator Category: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Race/ Ethnicity: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Place: string (nullable = true)



<H3>Counting Null Values

In [8]:
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------------+---------+----+------+---------------+-----+-----+
|Indicator Category|Indicator|Year|Gender|Race/ Ethnicity|Value|Place|
+------------------+---------+----+------+---------------+-----+-----+
|                 0|       28|  28|   218|            212|  231|  218|
+------------------+---------+----+------+---------------+-----+-----+



Since there are several null values in the columns as shown in the table above, first steps would be to remove / replace null values in each column

<H3>Working with Null Values

In [9]:
df.filter(df["Indicator"].isNull()).show(28)

+--------------------+---------+----+------+---------------+-----+-----+
|  Indicator Category|Indicator|Year|Gender|Race/ Ethnicity|Value|Place|
+--------------------+---------+----+------+---------------+-----+-----+
| FOR THE POPULATI...|     null|null|  null|           null| null| null|
|  12 MONTHS (S1701)"|     null|null|  null|           null| null| null|
|            (S1701)"|     null|null|  null|           null| null| null|
|            (S1701)"|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|           null| null| null|
|from the flu shot...|     null|null|  null|       

Since all the rows that have null values in Indicator have null values for other columns like Year, Gender, Race and etc, it would be better to remove these observations

In [10]:
# Counting total number of rows in the dataset to compare with the rows after null value rows are removed.
rows_count_pre = df.count()
print("Total number of rows before deleting: ",rows_count_pre)

Total number of rows before deleting:  13730


In [11]:
# deleting all the rows where there are null values in the columns mentioned below
df = df.na.drop(subset=["Indicator", "Year", "Gender", "Race/ Ethnicity", "Value", "Place"])

In [12]:
rows_count_post = df.count()
print("Total number of rows after deleting: ",rows_count_post)

Total number of rows after deleting:  13499


In [13]:
total_rows_removed = rows_count_pre - rows_count_post
print("Total number of rows deleted: ", total_rows_removed)

Total number of rows deleted:  231


In [14]:
#Checking the null values again to see if the dataset is clean
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------------+---------+----+------+---------------+-----+-----+
|Indicator Category|Indicator|Year|Gender|Race/ Ethnicity|Value|Place|
+------------------+---------+----+------+---------------+-----+-----+
|                 0|        0|   0|     0|              0|    0|    0|
+------------------+---------+----+------+---------------+-----+-----+



The results above show that all the rows with null values have been deleted from the dataset. This completes the step of removing all the null values from the dataset

<H3>Splitting the Place Column into City and State Columns

In [15]:
split_col = F.split(df["Place"], ',')
df = df.withColumn("City_County", split_col.getItem(0))
df = df.withColumn("State", split_col.getItem(1))
df.select("City_County", "State").show(truncate=False)

+-----------------------+-----+
|City_County            |State|
+-----------------------+-----+
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
|Atlanta (Fulton County)| GA  |
+-----------------------+-----+
only showing top 20 rows



In [ ]:
Creating a User Defined Function to take care of the City_County column to extract the city. Same can be done using

In [16]:
import re
def extract_city(city_str):
    result = re.sub(r'\([^)]*\)', '', city_str)
    return result

In [17]:
from pyspark.sql.types import StringType
udfExtract = F.udf(extract_city, StringType())
df = df.withColumn("City", udfExtract(df["City_County"]))
df.select("City", "State").show(truncate=False)

+--------+-----+
|City    |State|
+--------+-----+
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
|Atlanta | GA  |
+--------+-----+
only showing top 20 rows



This sums up the cleaning process of data using PySpark. Below is the final state of the dataset

In [27]:
df.show()

+--------------------+--------------------+----+------+---------------+-----+--------------------+--------------------+-----+--------+
|  Indicator Category|           Indicator|Year|Gender|Race/ Ethnicity|Value|               Place|         City_County|State|    City|
+--------------------+--------------------+----+------+---------------+-----+--------------------+--------------------+-----+--------+
|            HIV/AIDS|AIDS Diagnoses Ra...|2013|  Both|            All| 30.4|Atlanta (Fulton C...|Atlanta (Fulton C...|   GA|Atlanta |
|            HIV/AIDS|AIDS Diagnoses Ra...|2012|  Both|            All| 39.6|Atlanta (Fulton C...|Atlanta (Fulton C...|   GA|Atlanta |
|            HIV/AIDS|AIDS Diagnoses Ra...|2011|  Both|            All| 41.7|Atlanta (Fulton C...|Atlanta (Fulton C...|   GA|Atlanta |
|              Cancer|All Types of Canc...|2013|  Male|            All|195.8|Atlanta (Fulton C...|Atlanta (Fulton C...|   GA|Atlanta |
|              Cancer|All Types of Canc...|2013|Female|